<a href="https://colab.research.google.com/github/Zhichao-Zhang/6Rover/blob/main/6Rover%EF%BC%9A%E6%A8%A1%E5%BC%8F%E6%8F%90%E5%8F%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 环境、依赖、公共模块

In [1]:
!pip install IPy
!pip install pyasn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for IPy: filename=IPy-1.1-py3-none-any.whl size=19453 sha256=c207e2a35631317c24138cf00ec4d9d884a2821d4c7d623dbfc34854c2c97a2a
  Stored in directory: /root/.cache/pip/wheels/5e/18/5a/8645961e3a74758d77c2e53f9b0c7377ecf39dea238d343524
Successfully built IPy
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 KB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyasn: filename=pyasn-1.6.1-cp39-cp39-linux_x86_64.whl size=74980 sha256=64ceb5a9a9f63945ddc04e70c5e6bf9a93cea5f604c5bc2db548a381574dbedc
  Stored in directory: /root/.cache/pip/wheels/18/f4/57/f24b1f853566a33baa355dc2fe41facfd7283f7c092c01850c
Successfully built pyasn


In [2]:
from IPy import IP
import math
from google.colab import drive
import json
import matplotlib.pyplot as plt
import numpy as np
import pyasn


drive.mount('/content/drive')
DRIVE_DIR = "/content/drive/MyDrive/ipv6_seeds/"

asndb = pyasn.pyasn(DRIVE_DIR + 'ipasn_db.dat')

Mounted at /content/drive


# 模式提取算法

In [3]:
class Driver_data_loader():
  def __init__(self, dir = DRIVE_DIR):
    self.dir = dir
    
  def load_dict(self, sub_path):
    with open(self.dir + sub_path) as fp:
      res = json.load(fp)
    return res

  def load_hitlist(self, sub_path):
    addrSet = set()
    with open(self.dir + sub_path, "r") as f:
      lines = f.readlines()
      for line in lines:  
        addr = line.rstrip('\n')
        addrSet.add(addr)
    print("{} has {} active address".format(sub_path, len(addrSet)))
    return addrSet
  
  def covert_pfx_tb(self, addrSet):
    """load pfx2addrTb , as2addrTb"""
    pfx2addrTb = dict()
    as2addrTb = dict()
    # as2pfxTb = dict()
    for addr in addrSet:
      asn, pfx = asndb.lookup(addr)
      if asn not in as2addrTb:
        as2addrTb[asn] = set()
      if pfx not in pfx2addrTb:
        pfx2addrTb[pfx] = set()
      # if asn not in as2pfxTb:
      #   as2pfxTb[asn] = set()

      as2addrTb[asn].add(addr)
      pfx2addrTb[pfx].add(addr)
      # as2pfxTb[asn].add(pfx)
    # return pfx2addrTb, as2addrTb, as2pfxTb
    return pfx2addrTb , as2addrTb

  def load_allAsSet(self, fileName = DRIVE_DIR+ "ipasn_db.dat"):
    allAsSet = set()
    with open(fileName) as f:
        lines = f.readlines()
        for line in lines:
          line = line.rstrip()
          if line[0] != '2':
            continue
          record = line.split('	')
          allAsSet.add(record[1])
    return allAsSet


In [4]:
class IP_Converter():
  def __init__(self, pfx2addr):
    self.pfx2addr = pfx2addr
    self.pfx_to_addr_np = dict()

  def convert(self):
    counter = 0
    for pfx in self.pfx2addr:
      print("pfx: ", pfx)
      self.pfx_to_addr_np[pfx] = []
      for ip in self.pfx2addr[pfx]:
        temp = [int(x, 16) for x in IP(ip).strFullsize().replace(":", "")]       
        self.pfx_to_addr_np[pfx].append(temp)
      counter += len(self.pfx_to_addr_np[pfx])
      print(counter)


# 使用以下脚本以构建np数组
# with open(DRIVE_DIR + "pfx2addrTb-20230209_all.json") as f:
#   pfx2addr = json.load(f)
# ip_converter = IP_Converter(pfx2addr)
# ip_converter.convert()
# np.save(DRIVE_DIR +  'pfx_to_addr_np-20230211_all.npy', ip_converter.pfx_to_addr_np)


In [7]:
pfx_to_addr_np = np.load(DRIVE_DIR + "2.27/pfx_to_addr_np_20230314_all.npy", allow_pickle="True")

In [6]:
# len(pfx_to_addr_np.item()["2600:9000:20c3::/48"])

### utils: DHC address divide

In [ ]:
import queue

class Divider():
  def __init__(self):
    pass
  
  def dhc(self, arrs, mode = "leftmost"):
    if mode != "leftmost":
      print("Invalid mode")
      return 

    # q = queue.Queue()
    q = queue.LifoQueue()
    q.put(arrs)
    regions_arrs = []
    total_seed = len(arrs)
    while not q.empty():
        arrs = q.get()
        if len(arrs) < 100:
            regions_arrs.append(arrs)
            continue
        splits = self.leftmost(arrs)
        # break
        for s in splits:
            q.put(arrs[s])

    return regions_arrs

  def leftmost(self, arrs):
    Tarrs = arrs.T
    for i in range(32):
        splits = np.bincount(Tarrs[i], minlength=16)
        # print("splits: ", splits)

        if len(splits[splits > 0]) > 1:
            split_index = i
            split_nibbles = np.where(splits != 0)[0]
            break

    return [
        np.where(Tarrs[split_index] == nibble)[0] for nibble in split_nibbles
    ]

  def show_regions(self, arrs):
    address_space = []
    Tarrs = arrs.T
    for i in range(32):
        splits = np.bincount(Tarrs[i], minlength=16)
        # print(i, splits, np.argwhere(splits > 0)[0][0])
        if len(splits[splits > 0]) == 1:

            address_space.append(format(np.argwhere(splits > 0)[0][0], "x"))
        else:
            address_space.append("*")
        print(address_space)

    print("********Region**********")
    print("".join(address_space))
    for i in range(len(arrs)):
        print("".join([format(x, "x") for x in arrs[i]]), " ", i)

  def gene_partition(self, arrs, pattern_pfx_dict, label):
    address_space = []
    Tarrs = arrs.T
    for i in range(32):
      splits = np.bincount(Tarrs[i], minlength=16)
      # print(i, splits, np.argwhere(splits > 0)[0][0])
      if len(splits[splits > 0]) == 1:

          address_space.append(format(np.argwhere(splits > 0)[0][0], "x"))
      else:
          address_space.append("*")
        # print(address_space)

    # print("********Region**********")
    addr_pattern = "".join(address_space)
    rand_idx = addr_pattern.find("*")
    pfx = addr_pattern[: rand_idx] + "/" + str(4 * (rand_idx -1) ) +"_" + str(label)
    if pfx == "null" or pfx == None:
      print(address_space)
      print(arrs)

    if pfx in pattern_pfx_dict:
      pattern_pfx_dict[pfx].extend(arrs)
    else:
      pattern_pfx_dict[pfx] = arrs
    return pattern_pfx_dict

In [ ]:
d = Divider()
dhc_res = d.dhc(np.asarray(pfx_to_addr_np.item()["2600:9000:20c3::/48"]))


NameError: ignored

In [ ]:
for i in range(10):
  address_space = []
  arrs = dhc_res[i]
  Tarrs = arrs.T
  for i in range(32):
        splits = np.bincount(Tarrs[i], minlength=16)
        # print(i, splits, np.argwhere(splits > 0)[0][0])
        if len(splits[splits > 0]) == 1:

            address_space.append(format(np.argwhere(splits > 0)[0][0], "x"))
        else:
            address_space.append("*")
  print("".join(address_space) +"\n")

  for i in range(len(arrs)):
    print("".join([format(x, "x") for x in arrs[i]]), " ", i) 
  print("========================================")

## 1 创建模式
给定一组地址，提取模式

In [ ]:
from scipy.stats import entropy

class PatternExtractor():
  def __init__(self):
    # self.data_source = DRIVE_DIR + filename
    pass

  def entropy_cal(self, arrs):
    """ arrs """
    entropy_tb = [] 
    count_tb = []
    Tarrs = arrs.T
    for i in range(32):
      splits = np.bincount(Tarrs[i], minlength=16)
      count_tb.append(splits)
      if len(splits[splits > 0]) == 1:
        entropy_tb.append(0)
        continue
      p = splits[np.where(splits != 0)] / len(arrs)
      H = entropy(p, base = 16)
      entropy_tb.append(H)
      
    return entropy_tb, count_tb

  def extract(self, entropy_tb, count_tb, H_random = 0.8, range_random = 9):
  # 根据熵 & 每位计数，提取pattern
  #     # 算法：pattern包含3种元素，fixed random list；
  #     # 1. 当Entropy > 0.98， 记为random （*）
  #     # 2. 否则，遍历count_tb，取count最大值的符号作为“模式"，
  #     # 此外，重新计算熵值（底数-1），如果结果不是random，重复以上模式
    pattern = [] 
    for i in range(len(entropy_tb)):
      fixed_val_idx = np.where(count_tb[i] > 0)[0]
      # print(fixed_val_idx)
      if entropy_tb[i] == 0:
        if len(fixed_val_idx) == 1:
          # print(fixed_val_idx)
          pattern.append(format(fixed_val_idx[0], "x"))
        else:
          # print(fixed_val_idx)
          pattern.append([format(bit, "x") for bit in list(fixed_val_idx)])
        continue

      if entropy_tb[i] >= H_random and len(fixed_val_idx) >= range_random: #TODO 参数1
        # print("debug: ", len(fixed_val_idx), " ", i)
        pattern.append("*")
        continue

      else:
        average = np.sum(count_tb[i]) / 16
        above_baseline_idx = np.where(count_tb[i] > average)[0]
        if len(above_baseline_idx) == 1:
          pattern.append(format(above_baseline_idx[0], "x"))
        else:
          pattern.append([format(bit, "x") for bit in list(above_baseline_idx)])
    neat_pattern = "".join([str(p) for p in pattern])
    seperated_pattern = "_".join([str(p) for p in pattern]) # 方便进行切分
    # print("pattern: ", len(pattern))
    return neat_pattern, seperated_pattern, np.sum(count_tb[0])
  

In [ ]:
from sklearn.cluster import DBSCAN

def generate_pattern_res(pfx_to_addr_np):
  # 新增汉明距离的聚类
  d = Divider()
  pe = PatternExtractor()
  pattern_res = { key : dict() for key in pfx_to_addr_np.item()}
  counter = 0

  for key in pfx_to_addr_np.item():
    counter += len(pfx_to_addr_np.item()[key])
    print("key: {} , progress: {}".format(key, counter / 35000000 ))
    pattern_pfx_dict = dict()

    data = np.asarray(pfx_to_addr_np.item()[key])
    results = d.dhc(data)

    for r in results:
      if "null" in pattern_pfx_dict:
        print(r)
      # 汉明距离距离
      
      dbscan = DBSCAN(eps=20, min_samples=1).fit(r)
      # print(dbscan.labels_)

      # for i in range(len(r)):
      #   print("".join([format(x, "x") for x in r[i]]), " ", i) 
    
      groups = {}
      for i, label in enumerate(dbscan.labels_):
        if label not in groups:
          groups[label] = []
        groups[label].append(r[i])
    
      for label in groups:
        pattern_pfx_dict = d.gene_partition(np.asarray(groups[label]), pattern_pfx_dict, label)

    # print({ key : len(pattern_pfx_dict[key]) for key in pattern_pfx_dict})
    for ppfx in pattern_pfx_dict:
      entropy_tb, count_tb = pe.entropy_cal(pattern_pfx_dict[ppfx])
      pattern, seperated_pattern, seed_num = pe.extract(entropy_tb, count_tb )
      pattern_res[key][ppfx] = {"neat_pattern": pattern, "seperated_pattern": seperated_pattern, "seed_num": str(seed_num)}
      # print("seed num " + str(len(pattern_pfx_dict[ppfx])) + "  " + str(seed_num))
  
  return pattern_res


# temp = np.asarray({ "2804:a8::/32": pfx_to_addr_np.item()["2804:a8::/32"]})
# pattern_res = generate_pattern_res(temp)
pattern_res = generate_pattern_res(pfx_to_addr_np)


# for key in pfx_to_addr_np.item():
#   if key == "null":
#     print(pfx_to_addr_np.item()[key])

  # print({x: count_tb[x] for x in range(len(count_tb))})
  # print(pattern)
  # break
  # print("pfx: {} \npattern: {}\n".format(key, pattern))
  


流式输出内容被截断，只能显示最后 5000 行内容。
key: 2806:2f0:3362::/48 , progress: 1.0124623714285714
key: 2001:559:179::/48 , progress: 1.0124624
key: 2605:3380:45ff::/48 , progress: 1.0124624285714285
key: 2409:8734:5000::/37 , progress: 1.012462457142857
key: 2602:fc23:200::/40 , progress: 1.0124624857142857
key: 2408:8456:5420::/44 , progress: 1.0124625142857142
key: 2a02:e980:20b::/48 , progress: 1.0124625428571428
key: 2001:16d8:57::/48 , progress: 1.0124626
key: 2a00:15a8:600::/40 , progress: 1.0124626571428572
key: 2a03:600::/32 , progress: 1.0124627428571429
key: 2806:2f0:2142::/48 , progress: 1.0124627714285714
key: 2407:98c0::/32 , progress: 1.0124628285714286
key: 2800:c70:80::/44 , progress: 1.012462857142857
key: 2401:ff80:1012::/48 , progress: 1.0124629142857142
key: 2a11:b480::/29 , progress: 1.0124629428571428
key: 2a03:6a0::/32 , progress: 1.0124629714285713
key: 2a00:1728:18::/48 , progress: 1.012463
key: 2001:1248:9aed::/48 , progress: 1.0124630285714287
key: 2001:df1:1880::/48 , progr

In [ ]:
len(pattern_res)

62001

In [ ]:
pfx_to_addr_np = None

In [ ]:
#  with open(DRIVE_DIR +  '2.27/patternResult_20230316_all.json', 'w') as fp:
#   fp.write(json.dumps(pattern_res, indent=4)) 
# # dhc_res = d.dhc(np.asarray(pfx_to_addr_np.item()["2407:5200::/32"]))
# # address_space = []
# # arrs = dhc_res[1]
# # Tarrs = arrs.T
# # for i in range(32):
# #   splits = np.bincount(Tarrs[i], minlength=16)
# #   # print(i, splits, np.argwhere(splits > 0)[0][0])
# #   if len(splits[splits > 0]) == 1:

# #     address_space.append(format(np.argwhere(splits > 0)[0][0], "x"))
# #   else:
# #     address_space.append("*")
# # print("".join(address_space) +"\n")

# # for i in range(len(arrs)):
# #   print("".join([format(x, "x") for x in arrs[i]]), " ", i) 
# # print("========================================")

## 2 处理模式

In [8]:
with open(DRIVE_DIR +  '2.27/patternResult_20230316_all.json') as fp:
  pattern_res = json.load(fp) 

In [51]:
def show_one_pattern(idx): 
  bgp_num, total_pattern, filtered_pattern_num, seed_total_num ,seed_filter_num = 0,0, 0, 0, 0
  seed_thre = 10
  iter = 0
  for bgp in pattern_res:
    iter += 1 
    if iter != idx:
      continue
    
    
    bgp_num += 1
    total_pattern += len(pattern_res[bgp])
    for sub_pfx in pattern_res[bgp]:
      seed_total_num += int(pattern_res[bgp][sub_pfx]["seed_num"])
      if int(pattern_res[bgp][sub_pfx]["seed_num"]) < seed_thre:
        continue
      seed_filter_num += int(pattern_res[bgp][sub_pfx]["seed_num"])
      filtered_pattern_num += 1
      print(pattern_res[bgp][sub_pfx]["neat_pattern"] + "   " + pattern_res[bgp][sub_pfx]["seed_num"])
    break


  print(bgp)
  print(f"种子阈值量： {seed_thre}")

  print(f"过滤前的模式数量： {total_pattern}")
  print(f"过滤后的模式数量： {filtered_pattern_num}")


  print(f"过滤前的种子数量： {seed_total_num}")
  print(f"过滤后的种子数量： {seed_filter_num}")

def stat_pattern(seed_thre = 10):
  bgp_num, total_pattern, filtered_pattern_num, seed_total_num ,seed_filter_num = 0,0, 0, 0, 0
  for bgp in pattern_res:
    bgp_num += 1
    total_pattern += len(pattern_res[bgp])
    for sub_pfx in pattern_res[bgp]:
      seed_total_num += int(pattern_res[bgp][sub_pfx]["seed_num"])
      if int(pattern_res[bgp][sub_pfx]["seed_num"]) < seed_thre:
        continue
      seed_filter_num += int(pattern_res[bgp][sub_pfx]["seed_num"])
      filtered_pattern_num += 1

  print(f"bgp前缀数量：{bgp_num}")
  print(f"种子阈值量： {seed_thre}")

  print(f"过滤前的模式数量： {total_pattern}")
  print(f"过滤后的模式数量： {filtered_pattern_num}")

  print(f"过滤前的种子数量： {seed_total_num}")
  print(f"过滤后的种子数量： {seed_filter_num}")

In [52]:
stat_pattern()

bgp前缀数量：62001
种子阈值量： 10
过滤前的模式数量： 6202874
过滤后的模式数量： 1246565
过滤前的种子数量： 35441751
过滤后的种子数量： 26287026


In [53]:
show_one_pattern(6622)

2607f34810['0', '1', '3']['0', '4']['0', 'f']['0', 'f']000000000000000['0', '1']['0', '1', '2', '4', 'a']*   84
2607f348000['0', '1', '2', '4']0000000000000000000['1', '2', '5', '6']   16
2607:f348::/32
种子阈值量： 10
过滤前的模式数量： 5
过滤后的模式数量： 2
过滤前的种子数量： 108
过滤后的种子数量： 100


###  模式的实验1：分别提取pattern result的模式、子网
依据：DHC聚类结果中，前缀除了bgp宣告前缀之外，为子网

In [ ]:
class Pattern_Analyser():
  def __init__(self, pattern_res):
    self.pattern_res = pattern_res
    self.subnet_pattern_map = dict() # key：id（bgp_pfx）， val：entity（subnet_r: seed_num，pattern）

  def subnet_mapping(self):
    for bgp_pfx in self.pattern_res:
      # 将bgp_pfx格式化
      # print(bgp_pfx)
      if bgp_pfx == 'null':
        continue
      mask = int(bgp_pfx.split("/")[1]) // 4
      self.subnet_pattern_map[bgp_pfx] = dict()
      for nets in self.pattern_res[bgp_pfx]:
        long_subnet = nets.split("/")[0] # 作为唯一id
        routing_subnet = long_subnet[mask:]
        iid_pattern = self.pattern_res[bgp_pfx][nets]["pattern"][len(long_subnet):]
        seed_num = self.pattern_res[bgp_pfx][nets]["seed_num"]
        seperated_pattern = self.pattern_res[bgp_pfx][nets]["seperated_pattern"]

        
        seperated_pattern = "_".join(seperated_pattern.split("_")[len(long_subnet) :])
        if routing_subnet in self.subnet_pattern_map[bgp_pfx]:
          print("Repeated Subnet")

        self.subnet_pattern_map[bgp_pfx][routing_subnet] = {"pfx": long_subnet, "iid_pattern": iid_pattern, "seed_num": seed_num, "seperated_iid_pattern": seperated_pattern}
        # print(self.subnet_pattern_map[long_subnet])
    return self.subnet_pattern_map

pa = Pattern_Analyser(pattern_result)
subnet_pattern_map = pa.subnet_mapping()


  


In [ ]:
subnet_pattern_map
# with open(DRIVE_DIR + "subnet_pattern_map_0214_all.json", "w") as f:
#   f.write(json.dumps(subnet_pattern_map, indent=4)) 

DHC前缀聚类后，过少的种子区域占比很少，可以直接忽略

In [ ]:
counter= 0
total = 0
for key in subnet_pattern_map:
  for key2 in subnet_pattern_map[key]:
    total += 1
    if int(subnet_pattern_map[key][key2]["seed_num"]) <=  10:
      counter += 1
    # counter += int(subnet_pattern_map[key][key2]["seed_num"])

In [ ]:
counter / total

0.3008561281369805

### 计算模式相似度 + 聚类


In [ ]:
with open(DRIVE_DIR +  'subnet_pattern_map_0214_all.json') as fp:
  subnet_pattern_map = json.load(fp)

In [ ]:
import ast

class Similarity_Cluster():
  def __init__(self):
    self.pattern_entities = []
  
  def preprocess(self, subnet_pattern_map, min_seed_num = 10):
    counter = {"total_seed": 0, "dropped_seed": 0}
    for bgppfx in subnet_pattern_map:
      for subnet in subnet_pattern_map[bgppfx]:
        counter["total_seed"] += 1
        if int(subnet_pattern_map[bgppfx][subnet]["seed_num"]) <= min_seed_num:
          counter["dropped_seed"] += 1
          continue
        temp = subnet_pattern_map[bgppfx][subnet]
        temp["bgp_pfx"] = bgppfx
        temp["subnet"] = subnet
        self.pattern_entities.append(temp)
    print("percentage of dropped seeds: {:.1%}".format(counter["dropped_seed"] / counter["total_seed"]))


  def distance(self, i, j):
    pe1 = self.pattern_entities[i]["seperated_iid_pattern"].split("_")
    pe2 = self.pattern_entities[j]["seperated_iid_pattern"].split("_")
    diff = 0
    for idx in range( -1,  - min(len(pe1), len(pe2)), -1):
      # 倒叙遍历
      diff += self.pointwise_distance(pe1[idx], pe2[idx])
    return diff
  
  def pointwise_distance(self, p1, p2):
    if p1 == "*" and p2[0] == "*":
      return 0 
    if p1 == "*" and p2[0] != "[": #random and fixed
      return 15
    if p1 == "*" and p2[0] == "[":
      return 16 - len(ast.literal_eval(p2))
    
    if p1[0] == "[" and p2[0] == "*":
      return 16 - len(ast.literal_eval(p1))

    if p1[0] == "[" and p2[0] != "[":
      if p2 in ast.literal_eval(p1):
        return len(ast.literal_eval(p1)) - 1
      else:
        return len(ast.literal_eval(p1))

    if p1[0] == "[" and p2[0] == "[":
      p1_set = set(ast.literal_eval(p1))
      p2_set = set(ast.literal_eval(p2))
      union = p1_set.union(p2_set)
      intersec = p1_set.intersection(p2_set)
      return len(union) - len(intersec)

    if p1[0] != "[" and p2[0] == "*":
      return 15
    if p1[0] != "[" and p2[0] != "[":
      if p1 == p2:
        return 0
      else:
        return 1
    if p1[0] != "[" and p2[0] == "[":
      if p1 in ast.literal_eval(p2):
        return len(ast.literal_eval(p2)) - 1
      else:
        return len(ast.literal_eval(p2))

  def generate_distance_matrix(self):
    dis_matrix = np.empty([len(self.pattern_entities), len(self.pattern_entities)] , dtype=int)
    for i in range(len(self.pattern_entities)):
      for j in range(i, len(self.pattern_entities)):
        if i == j:
          dis_matrix[i, j] = 0
        else:
          dis_matrix[i, j] = self.distance(i, j)
          dis_matrix[j, i] = dis_matrix[i, j] 
    return dis_matrix
  




In [ ]:
sc = Similarity_Cluster()
temp_map = dict()
temp_map["240e:380::/29"] = subnet_pattern_map["240e:380::/29"]
sc.preprocess(temp_map)

percentage of dropped seeds: 14.1%


In [ ]:
len(sc.pattern_entities)

413

In [ ]:
dis_matrix = sc.generate_distance_matrix()

In [ ]:
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import euclidean_distances

db = DBSCAN(metric="precomputed", eps=.5, min_samples=2).fit(dis_matrix)
labels = db.labels_

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, random_state=0, n_init=10).fit(dis_matrix)

In [ ]:
kmeans.labels_

array([8, 2, 2, 4, 1, 2, 2, 2, 2, 8, 4, 2, 2, 1, 4, 5, 4, 9, 4, 8, 8, 2,
       2, 1, 8, 2, 2, 4, 2, 8, 2, 1, 1, 4, 4, 1, 4, 4, 8, 8, 4, 4, 4, 4,
       4, 4, 4, 4, 8, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 8, 4, 2, 1, 9,
       1, 1, 9, 9, 9, 3, 4, 4, 4, 1, 2, 2, 8, 4, 8, 8, 2, 8, 2, 2, 1, 1,
       4, 3, 8, 3, 8, 3, 3, 5, 0, 0, 6, 0, 6, 0, 0, 0, 0, 8, 6, 8, 7, 0,
       3, 6, 6, 0, 7, 7, 7, 7, 1, 2, 6, 4, 7, 4, 9, 7, 6, 6, 7, 7, 0, 7,
       0, 7, 1, 5, 0, 0, 0, 0, 7, 0, 9, 0, 4, 3, 4, 5, 7, 6, 6, 6, 6, 6,
       6, 6, 6, 0, 1, 8, 7, 5, 0, 7, 0, 7, 1, 3, 8, 7, 9, 8, 7, 0, 7, 7,
       0, 7, 7, 7, 7, 7, 7, 8, 0, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 0, 9,
       7, 0, 7, 7, 7, 7, 7, 7, 7, 7, 2, 7, 1, 1, 6, 7, 0, 7, 7, 7, 7, 4,
       8, 5, 4, 9, 5, 3, 9, 6, 8, 5, 9, 6, 7, 6, 0, 7, 7, 7, 8, 3, 7, 0,
       7, 7, 0, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 3, 7, 7,
       0, 6, 0, 0, 0, 3, 7, 7, 7, 7, 7, 7, 1, 1, 7, 8, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 8, 3, 7, 7, 7, 7, 7, 0, 7, 7, 0,

In [ ]:
kmeans.cluster_centers_.shape

(10, 413)

In [ ]:
[sc.pattern_entities[i]["subnet"] for i in range(len(labels)) if labels[i] == 2 ]

['195', '1914', '18d5', '18d4', '18d3', '18d2']

In [ ]:
dis_matrix[100,:]

array([194, 212, 206, 131, 163, 223, 223, 223, 223, 188, 129, 204, 205,
       152, 130,  60, 128, 151, 112, 195, 191, 218, 223, 163, 193, 226,
       200, 133, 224, 183, 225, 147, 157, 116, 120, 144, 123, 126, 197,
       184, 134, 111, 126, 123, 118, 118, 119, 122, 186, 159, 119, 111,
       109, 126, 126, 126, 126, 121, 121, 117, 117, 170, 121, 219, 164,
       137, 172, 155, 138, 123, 154,  95, 112, 115, 121, 163, 208, 223,
       202, 127, 198, 197, 206, 182, 220, 223, 165, 139, 140, 127, 183,
       142, 180, 113, 104,  66,  31,  34,  23,  26,   0,  34,  13,  33,
        26, 185,  22, 178,  40,  31, 105,  37,  38,  37,  40,  40,  40,
        40, 169, 208,  25, 137,  40, 112, 124,  40,  33,  43,  41,  40,
        28,  40,  35,  40, 151,  86,  36,  33,  36,  33,  40,  35, 119,
        26, 121, 129, 124, 127,  40,  40,  40,  39,  41,  37,  41,  40,
        37,  37, 160, 163,  40,  69,  34,  40,  35,  40, 146, 121, 191,
        41, 131, 193,  40,  36,  40,  47,  37,  40,  40,  40,  4

In [ ]:
[sc.pattern_entities[i]["subnet"] for i in range(len(labels)) if labels[i] == 3 ]

['0c20',
 '0be003',
 '0be002',
 '0be001',
 '0be000',
 '0a',
 '08c03a',
 '08c035',
 '08c032',
 '08c030',
 '08c001',
 '06e0',
 '056015',
 '056012',
 '056010',
 '048002',
 '048000',
 '044025',
 '044024',
 '044023',
 '044022',
 '044021',
 '044020',
 '03c028',
 '03c027',
 '03c026',
 '03c025',
 '03c024',
 '03c023',
 '03c022',
 '03c021',
 '03c020',
 '03c00',
 '038009',
 '038007',
 '038006',
 '038005',
 '038004',
 '038003',
 '038002',
 '038001',
 '038000',
 '0340',
 '032005',
 '032002',
 '032001',
 '032000',
 '010005',
 '010002',
 '010001',
 '010000',
 '00e05',
 '00e045',
 '00e044',
 '00e042',
 '00e041',
 '00e040',
 '00e034',
 '00e033',
 '00e032',
 '00e031',
 '00e030',
 '00e02',
 '00e016',
 '00e015',
 '00e014',
 '00e013',
 '00e012',
 '00e011',
 '00e010',
 '00c056',
 '00c055',
 '00c025',
 '00c024',
 '00c023',
 '00c022',
 '00c021',
 '00c020',
 '0080d',
 '0080b5',
 '0080b4',
 '0080b3',
 '0080b2',
 '0080b1',
 '0080b0',
 '0080a3',
 '0080a2',
 '0080a1',
 '0080a0',
 '008088',
 '008087',
 '008086',
 '

# HMap6数据分析

1. 种子地址：新增多少？
```
总种子数：35441751 ，交集个数：3981266, Gasser占比：66.5%, hmap6占比:44.8%
```
交集个数只有4百万，说明**HMap6发现了不少Gasser之外的地址**

2. 前缀：覆盖了多少？
```
前缀占比统计
 HMap6 /80: 前缀 : 21.0%, AS : 51.3% 
 全部种子：  前缀 : 35.3%, AS : 63.7% 
```

In [ ]:
# 加载数据
ddl = Driver_data_loader()
gasser_set = ddl.load_hitlist("active/addrSet_20221124.txt")
hmap6_set = ddl.load_hitlist("6HMap/Global_ICMPv6_20220424.txt")

active/addrSet_20221124.txt has 23560224 active address
6HMap/Global_ICMPv6_20220424.txt has 15862793 active address


In [ ]:
# 种子数比对
all_seed_set = gasser_set.union(hmap6_set)
seed_intersec = gasser_set.intersection(hmap6_set)
print("总种子数：{} ，交集个数：{}, Gasser占比：{:.1%}, hmap6占比:{:.1%}"
.format(len(all_seed_set), len(seed_intersec), len(gasser_set) / len(all_seed_set), len(hmap6_set)/ len(all_seed_set)))

总种子数：35441751 ，交集个数：3981266, Gasser占比：66.5%, hmap6占比:44.8%


In [ ]:
# 前缀覆盖率
hmap6_pfx_set = ddl.load_hitlist("6HMap/Global_80Prefix_20220610.txt")

6HMap/Global_80Prefix_20220610.txt has 31792206 active address


In [ ]:
as_set = ddl.load_allAsSet()

In [ ]:
pfx_tb_80pfx, as_tb_80pfx = ddl.covert_pfx_tb(hmap6_pfx_set)
pfx_tb_seed, as_tb_seed = ddl.covert_pfx_tb(all_seed_set)

In [ ]:
# print("前缀占比统计\n HMap6 /80: 前缀 : {:.1%}, AS : {:.1%} \n"
# .format(len(pfx_tb_80pfx) / 175734, len(as_tb_80pfx) / len(as_set)))

# print("全部种子  前缀 : {:.1%}, AS : {:.1%} \n"
# .format(len(pfx_tb_seed) / 175734, len(as_tb_seed) / len(as_set)))


前缀占比统计
 HMap6 /80: 前缀 : 21.0%, AS : 51.3% 

全部种子  前缀 : 35.3%, AS : 63.7% 



In [ ]:
# # 存储结果到本地，不需要每次都运行
# with open(DRIVE_DIR +  '2.27/as2addrTb_20230227_hmap6_80.json', 'w') as fp:
#     as2addrTb = {key : sorted(list(as_tb_80pfx[key])) for key in as_tb_80pfx}
#     fp.write(json.dumps(as2addrTb, indent=4))

# with open(DRIVE_DIR +  '2.27/pfx2addrTb_20230227_hmap6_80.json', 'w') as fp:
#   pfx2addrTb = {key : sorted(list(pfx_tb_80pfx[key])) for key in pfx_tb_80pfx}
#   fp.write(json.dumps(pfx2addrTb,  indent=4))

# 3.14模式提取实验
1. 网段在不同前缀下有相似性吗？

In [ ]:
def download_and_convert():
  # 导入pfx2addr dict
  ddl = Driver_data_loader()
  pfx2addr_all_dict = ddl.load_dict("2.27/pfx2addrTb_20230227_all.json")
  ipc = IP_Converter(pfx2addr_all_dict)
  ipc.convert()
  # 使用以下脚本以构建np数组
  # with open(DRIVE_DIR + "pfx2addrTb-20230209_all.json") as f:
  #   pfx2addr = json.load(f)
  # ip_converter = IP_Converter(pfx2addr)
  # ip_converter.convert()
  # np.save(DRIVE_DIR +  '2.27/pfx_to_addr_np_20230314_all.npy', 
  #         ipc.pfx_to_addr_np)

流式输出内容被截断，只能显示最后 5000 行内容。
pfx:  2001:1218:6041::/48
35439130
pfx:  2a00:1d58:f802::/48
35439131
pfx:  2a03:7380:180::/42
35439132
pfx:  2607:f038:401b::/48
35439133
pfx:  2401:4900:33b8::/48
35439134
pfx:  2606:bc00:bef0::/44
35439135
pfx:  2001:1248:9f08::/48
35439136
pfx:  2800:160:1b9b::/48
35439137
pfx:  2001:678:22c::/48
35439138
pfx:  2800:300:8330::/44
35439139
pfx:  2600:1410:c000::/48
35439140
pfx:  2607:ffb0:4002::/48
35439141
pfx:  2a03:b8c0:12::/48
35439142
pfx:  2001:49f0:d10a::/48
35439143
pfx:  240e:44d:1140::/42
35439144
pfx:  2a09:bac0:451::/48
35439145
pfx:  2606:5240::/32
35439146
pfx:  2409:8904:2d70::/44
35439147
pfx:  2405:7f00:ae00::/42
35439148
pfx:  2a02:e0:3808::/48
35439149
pfx:  2408:8756:c90::/44
35439150
pfx:  2a0c:f080::/30
35439151
pfx:  2001:c38:9013::/48
35439152
pfx:  2804:41d0:3200::/40
35439153
pfx:  2a0b:f304:404::/48
35439154
pfx:  2804:47a8::/32
35439155
pfx:  2804:9d4:1000::/48
35439156
pfx:  2a07:140::/32
35439157
pfx:  2800:160:1b8d::/48
3543

In [ ]:
list(ipc.pfx_to_addr_np.keys())
def load_np_dict(): 
  pfx_to_addr_np = np.load(DRIVE_DIR + "pfx_to_addr_np-20230211_all.npy", allow_pickle="True")
  return pfx_to_addr_np

['2407:5200::/32',
 '2a01:e0a::/38',
 '2600:9000:223e::/48',
 '2a02:2488::/32',
 '2408:8206:4000::/34',
 '2a02:c7f:8000::/33',
 '2804:a8::/32',
 '2804:1a04::/32',
 '240e:378::/29',
 '2a02:27b0:4400::/40',
 '240e:378::/36',
 '2003::/19',
 '2407:5200:300::/48',
 '2a01:e0a::/39',
 '2a02:26f0:3400::/48',
 '240e:1:8000::/33',
 '2600:9000:21a1::/48',
 '240e:38f::/32',
 '2a01:230:2::/48',
 '2001:ee0:4680::/42',
 '2a02:a400::/25',
 '2400:8500:1000::/36',
 '2a04:6ec0::/32',
 '2001:fb1::/32',
 '2a01:368::/29',
 '2a01:e00::/26',
 '2a01:7e00::/32',
 '2a02:e980:ab::/48',
 '240e:388::/32',
 '2a02:e981:f::/48',
 '2600:9000:20c3::/48',
 '2600:9000:21f3::/48',
 '2a00:6020::/32',
 '2a02:6d40::/32',
 '2001:418::/32',
 '2a01:e34::/32',
 '2600:3c00::/32',
 '2600:3c01::/32',
 '2a00:1288:110::/48',
 '2408:8710::/32',
 '2a01:e0a:400::/39',
 '2804:298::/32',
 '2a02:908::/33',
 '2a03:80c0:1::/48',
 '2620:108:7000::/44',
 '2804:d40::/28',
 '2001:ee0:48c0::/42',
 '2001:ee0:4700::/42',
 '2600:9000:20eb::/48',
 '24

In [ ]:
pe = PatternExtractor()
pe.entropy_cal(np.asarray(ipc.pfx_to_addr_np['2409:8c00::/32']))

([0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0.14505217398412745,
  0.1270528761948504,
  0.19423531656339996,
  0.19851257951309748,
  0.17239407053435896,
  0.05040179545115335,
  0.1698576468033363,
  0.2554073443083875,
  0.012787107255841968,
  0.032172384195112966,
  0.07717492544946795,
  0.09512796704981513,
  0.006394722389046713,
  0.006394722389046713,
  0.03812934101790456,
  0.03812934101790456,
  0.1631000766932792,
  0.020485964307241762,
  0.18429527494132936,
  0.18212915842189348,
  0.8626554287661145,
  0.9400111287612447,
  0.9711638050524393,
  0.9842587537385137],
 [array([   0,    0, 5113,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0]),
  array([   0,    0,    0,    0, 5113,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0]),
  array([5113,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0]),
  array([   0,    0,    0,    0,    0,    0,    0,    0,    0,

# 模式提取

In [ ]:
with open(DRIVE_DIR +  '2.27/patternResult_20230316_all.json') as f:
  pattern_res = json.load(f)